# NLP with Multinomial Classifier

*   Using embeddings of k-mers, then fed on multinomial classifier.
*   Training accuracy is 99.5%.
*   The final Kaggle accuracy is 96.990%.(Due to entry issue, I could not submit this result successfully at the due date.)



In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df_train = pd.read_csv (r'/content/train_features.csv')
df_test = pd.read_csv (r'/content/test_features.csv')
df_label = pd.read_csv (r'/content/train_labels.csv')

y_train = np.array(df_label['labels'], dtype = "uint32")

In [4]:
df_combine = df_train.append(df_test, ignore_index = True)
df_combine.shape

(21212, 2)

In [5]:

def Kmers_funct(seq, size=6):
   return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]
#convert our training data sequences into short overlapping k-mers of length 6. 
dna = df_combine['dna']
x_word= dna.apply(lambda x: Kmers_funct(x))



In [16]:
x_word

0        [aacatt, acatta, cattat, attata, ttatac, tatac...
1        [tacact, acacta, cactat, actata, ctatac, tatac...
2        [------, ------, ------, ------, ------, -----...
3        [aacatt, acatta, cattat, attata, ttatat, tatat...
4        [aacttt, acttta, ctttat, tttata, ttatat, tatat...
                               ...                        
21207    [------, ------, ------, ------, ------, -----...
21208    [tacatt, acatta, cattat, attata, ttatat, tatat...
21209    [aacatt, acatta, cattat, attata, ttatat, tatat...
21210    [tacttt, acttta, ctttat, tttata, ttatat, tatat...
21211    [gacttt, actttg, ctttgt, tttgta, ttgtat, tgtat...
Name: dna, Length: 21212, dtype: object

In [6]:
y = pandas.DataFrame(y_train)
x_texts = list(x_word)
for item in range(len(x_texts)):
   x_texts[item] = ' '.join(x_texts[item])
y_label = y.iloc[:, 0].values

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4)) #The n-gram size of 4 is previously determined by testing
X = cv.fit_transform(x_texts)

In [9]:
subX = X[0:12906,]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(subX,
                                                   y_label,
                                                   test_size = 0.20,
                                                   random_state=42)
                                                   
### Multinomial Naive Bayes Classifier ###
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(subX, y_label)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [12]:
y_pred = classifier.predict(X_test)

In [13]:
y_pred

array([1212, 1062,  327, ...,  624,   77,  886], dtype=uint32)

In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print("Confusion matrix for predictions on human test DNA sequence\n")

print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))

def get_metrics(y_test, y_predicted):
   accuracy = accuracy_score(y_test, y_predicted)
   precision = precision_score(y_test, y_predicted, average='weighted')
   recall = recall_score(y_test, y_predicted, average='weighted')
   f1 = f1_score(y_test, y_predicted, average='weighted')
   return accuracy, precision, recall, f1
   
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)

print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix for predictions on human test DNA sequence

Predicted  1     2     3     4     5     ...  1207  1209  1211  1212  1213
Actual                                   ...                              
1             2     0     0     0     0  ...     0     0     0     0     0
2             0     1     0     0     0  ...     0     0     0     0     0
3             0     0     3     0     0  ...     0     0     0     0     0
4             0     0     0     3     0  ...     0     0     0     0     0
5             0     0     0     0     5  ...     0     0     0     0     0
...         ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...
1207          0     0     0     0     0  ...     1     0     0     0     0
1209          0     0     0     0     0  ...     0     2     0     0     0
1211          0     0     0     0     0  ...     0     0     3     0     0
1212          0     0     0     0     0  ...     0     0     0    19     0
1213          0     0     0     0     0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
df_test.shape

(8306, 2)

In [ ]:
y_submit = classifier.predict(X[12906:21212, ])

In [ ]:
y_submit.shape

(8306,)

In [ ]:

np.savetxt("pred_nlp3.csv", y_submit)